# Book-Oracle

Book-Oracle is a book recommendation apps which suggest books based on preferences.

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

#modelling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from scipy.stats import loguniform
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from transformers import pipeline
from tqdm import tqdm
import nltk


#plotting
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter
plt.rcParams.update({ "figure.figsize" : (8, 5),"axes.facecolor" : "white", "axes.edgecolor":  "black"})
plt.rcParams["figure.facecolor"]= "w"
pd.plotting.register_matplotlib_converters()
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.float_format = "{:,.2f}".format

import warnings
warnings.filterwarnings('ignore')

RSEED = 42

## Import Data

In [2]:
# Import data for ratings csv file
df_ratings = pd.read_csv("data/Ratings.csv")

In [ ]:
#Checking no of rows and columns in dataframe
df_ratings.shape

In [ ]:
#Descriptive Stats 
df_ratings.describe()

In [ ]:
#Checking datatypes of columns in ratings dataframe
df_ratings.info()

(ISBN) is a 13-digit number that uniquely identifies books and book-like products published internationally.

In [37]:
#Checking the string length of ISBN cell values
ISBNCount = df_ratings['isbn'].str.len()
print(ISBNCount.unique())
# There are 7 unique ISBN values

[10  9 11  8 12 13  7  4  6]


In [ ]:
#Total rows with All ISBN's = 1149780
#Count of ISBN's with str(len) as 13 = 1740
#Count of ISBN's with str(len) as 10 = 1139363
#Count of ISBN's with str(len) as 9 = 5140
df_13 = df_ratings[df_ratings['ISBN'].str.len()==9]

In [ ]:
#This function provides the boolean value for the complete dataset to know if any null value is present or not.
df_ratings.isna().any()

In [ ]:
#This function gives the sum of the null values preset in the dataset column-wise.
df_ratings.isna().sum()

In [ ]:
df_ratings.duplicated().sum()

In [ ]:
#Print duplicate rows if any
duplicate_rows = df_ratings[df_ratings.duplicated(keep=False)]
print(duplicate_rows)

In [ ]:
print(df_ratings['User-ID'].nunique()) #105283
print(df_ratings['ISBN'].nunique()) #340556

In [ ]:
# Get count duplicates multiple columns using dataframe.pivot_table() 
#Here one User-ID has given ratings for MORE than ONE ISBN(book)

df2 = df_ratings.pivot_table(index = ['User-ID', 'ISBN'], aggfunc ='size')
print(df2)

NOTE : One User can rate multiple books (ISBN) likewise a Book(ISBN) can have ratings from multiple Users

Its M-M relationship


So there are NO duplicate entries in Ratings table

In [ ]:
#Get Count Duplicates for Each Unique Row
df2 = df_ratings.groupby(df_ratings.columns.tolist(), as_index=False).size()

#There are NO duplicates for unique combination of User-ID and ISBN
print(df2[df2['size'] > 1])

In [ ]:
#check if one ISBN has multiple UserID's

isbn_counts = df_ratings.groupby('ISBN').size()

# Filter ISBNs with multiple UserIDs
multiple_userids_isbn = isbn_counts[isbn_counts > 1]

# Display ISBNs with multiple UserIDs
print(multiple_userids_isbn)



Python function for data cleaning of Ratings csv file

- Renaming Columns
- Checking for duplicates if any
- Checking missing / null values

In [30]:
#Perform data cleaning operations on a DataFrame using pandas built-in functions.

def datacleaning_rating(df_ratings):
    
    # Rename columns
    column_name_mapping = {
    'User-ID': 'user_id',
    'ISBN': 'isbn',
    'Book-Rating': 'book_rating',
    # Add more mappings as needed
}
# Use the rename method to rename the columns
    df_ratings.rename(columns=column_name_mapping, inplace=True)
    print(df_ratings.columns)

    # Check for and display duplicate rows
    duplicate_rows = df_ratings[df_ratings.duplicated(keep=False)]
    print("Duplicate Rows:")
    print(duplicate_rows)

    # Check for and display rows with missing values
    missing_values = df_ratings[df_ratings.isna().any(axis=1)]
    print("\nRows with Missing Values:")
    print(missing_values)

    return df_ratings

In [ ]:
datacleaning_rating(df_ratings)

- Checking if ISBN has special characters or not

In [ ]:
## checking ISBN
import re
flag = 0
k =[]
reg = "[^A-Za-z0-9]"

for x in df_ratings['isbn']:
    z = re.search(reg,x)    
    if z:
        flag = 1

if flag == 1:
    print("False")
else:
    print("True")

- Removing special characters from ratings.ISBN column

In [25]:
## removing extra characters from ISBN (from ratings dataset)
ratingISBN = df_ratings['isbn'].tolist() 
reg = "[^A-Za-z0-9]" 
# Use a for loop to iterate over each row in the DataFrame
for index, row in df_ratings.iterrows():
    # Apply the regular expression to the 'ISBN' column
    cleaned_isbn = re.sub(reg, "", row['isbn'])
    # Update the 'ISBN' column with the cleaned value
    df_ratings.at[index, 'isbn'] = cleaned_isbn

## Data Cleaning for Each Table

## Exploratory Data Analysis for Each Table

### Statistical Summary

In [ ]:
df_ratings.describe(include='all')

- separate Numerical and categorical variables for easy analysis

In [ ]:
cat_cols=df_ratings.select_dtypes(include=['object']).columns
num_cols = df_ratings.select_dtypes(include=np.number).columns.tolist()
print("Categorical Variables:")
print(cat_cols)
print("Numerical Variables:")
print(num_cols)

- EDA Univariate Analysis


In [ ]:
for col in num_cols:
    print(col)
    print('Skew :', round(df_ratings[col].skew(), 2))
    plt.figure(figsize = (15, 4))
    plt.subplot(1, 2, 1)
    df_ratings[col].hist(grid=False)
    plt.ylabel('count')
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df_ratings[col])
    plt.show()

- Data Visulation based on book ratings ( 0 rating included)

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x="book_rating", data=df_ratings)

In [ ]:
## Explicit Ratings (Excluded rating 0 for better visualization)
plt.figure(figsize=(8,6))
data = df_ratings[df_ratings['book_rating'] != 0]
sns.countplot(x="book_rating", data=data)
plt.title("Explicit Ratings")

In [ ]:
#Description of book rating
df_ratings['book_rating'].describe()

### Observation - Book-Ratings are  distributed with median rating of 3

In [ ]:
plt.figure(figsize=(13,17))
sns.pairplot(data=df_ratings.drop(['isbn'],axis=1))
plt.show()

In [33]:
#Selecting Books with Optimum Number of Ratings (>5)

# counting the ratings >=5 by grouping ISBN and sort by ratings in desc order
isbn_rating_desc = df_ratings[df_ratings['book_rating'] >= 5].groupby(['isbn'])['book_rating'].count().reset_index().sort_values('book_rating', ascending=False)
print(isbn_rating_desc)

              isbn  book_rating
25486   0316666343          692
43466   0385504209          475
21627   0312195516          369
87465   0679781587          329
5087    0060928336          312
...            ...          ...
75387   0590313126            1
75386   0590312758            1
75385   059031257X            1
75384   0590312464            1
179221  ooo7156103            1

[179222 rows x 2 columns]


In [ ]:
#Considering all the counts greater than 5
rating_5plus=isbn_rating_desc[isbn_rating_desc['book_rating']>5]['isbn']
print(rating_5plus)

In [ ]:
# Convert 'ISBN' to category type for better visualization
df_ratings['ISBN'] = df_ratings['ISBN'].astype('category')

# Create a pair plot
sns.pairplot(df_ratings, hue='ISBN', markers='o', palette='viridis')

plt.show()

## Merge Tables

## Data Cleaning

## Exploratory Data Analysis

## Pipline Architecture

## Sample Size

## Modelling

## Evaluation

## Error Analysis